# Instalar dependencias

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
%load_ext tensorboard
import datetime, os

In [5]:
!pip install accelerate>=0.12.0 datasets>=1.8.0 sentencepiece!=0.1.92 scipy scikit-learn protobuf torch>=1.3 transformers evaluate

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.


# Usando modelos pre-entrenados de HuggingFace

Como libreria backbone emplearemos Transformers, que a su vez extiende pytorch.

Modelo a usar: Qwen2.0-0.5 Instruct
* basado en Qwen, una modelo opensource de la misma arquitectura que GPT-3
* entrenado con texto en 27 lenguajes

In [6]:
from transformers import pipeline, set_seed

# generator = pipeline('text-generation', model='nlpcloud/instruct-gpt-j-fp16')
generator = pipeline('text-generation', model='Qwen/Qwen2-0.5B-Instruct')

set_seed(42)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
output = generator("Explicame como se mueve la tierra alrededor del sol", max_length=200)
print(output[0]["generated_text"])

Explicame como se mueve la tierra alrededor del sol. La Tierra y el Sol están en un ángulo de aproximadamente 90 grados, lo que significa que cada día, el Sol pasa por un rango de 180 grados, pero el movimiento de la Tierra es hacia el Sol a medida que hace una circunferencia de aproximadamente 360 grados.

Por lo tanto, si consideramos que cada día la Tierra sale de la Luna a 24 horas, entonces la Tierra está moviendo hacia el Sol a medida que hace 360 grados (360/24 = 15) días.

Este movimiento constante de la Tierra hacia el Sol no es completamente impredecible debido a la influencia de los planetas celestes, incluyendo la estrella Neptúnico y la estrella Júpiter, pero generalmente


# Tecnicas de Prompting

## Zero-shot

In [10]:
output = generator("""
Descripcion: Clasifica el texto en Positivo o Negativo
Texto: Me encanto el producto.
Respuesta:
""", max_new_tokens=4, temperature=0.7)
print(output[0]["generated_text"])


Descripcion: Clasifica el texto en Positivo o Negativo
Texto: Me encanto el producto.
Respuesta:
Positiva.


In [11]:
output = generator("""
Descripcion: Clasifica el texto en Positivo o Negativo
Texto: Terrible producto, una perdidad de dinero.
Respuesta:
""", max_new_tokens=4, temperature=0.7)
print(output[0]["generated_text"])


Descripcion: Clasifica el texto en Positivo o Negativo
Texto: Terrible producto, una perdidad de dinero.
Respuesta:
Positiva




## Few-shot

In [12]:
output = generator("""
Descripcion: Clasifica el texto en Positivo o Negativo
Texto: Buen precio y buen servicio.
Respuesta: Positivo

Texto: Muy caro y mala comida.
Respuesta: Negativo

Texto: El ambiente es agradable y la comida es deliciosa.
""", max_new_tokens=10, temperature=0.7)
print(output[0]["generated_text"])


Descripcion: Clasifica el texto en Positivo o Negativo
Texto: Buen precio y buen servicio.
Respuesta: Positivo

Texto: Muy caro y mala comida.
Respuesta: Negtivo

Texto: El ambiente es agradable y la comida es deliciosa.
Respuesta: Positivo

Texto: La


In [16]:
output = generator("""
Descripcion: Clasifica el texto en Positivo o Negativo
Texto: Buen precio y buen servicio.
Respuesta: Positivo

Texto: Muy caro y mala comida.
Respuesta: Negativo

Texto: El ambiente es desagradable y la comida es mala.
""", max_new_tokens=10, temperature=0.7)
print(output[0]["generated_text"])


Descripcion: Clasifica el texto en Positivo o Negativo
Texto: Buen precio y buen servicio.
Respuesta: Positivo

Texto: Muy caro y mala comida.
Respuesta: Negativo

Texto: El ambiente es desagradable y la comida es mala.
Respuesta: Negativo

Texto: La comida


## Chain of Thought

La idea principal es pedirle al modelo que explique su razonamiento en pasos.

In [14]:
output = generator("""
Q: Roger tiene 5 pelotas. El compra 2 cajas de pelotas. Cada caja tiene 3 pelotas. ¿Cuántos pelotas tiene Roger?
A: La respuesta es 11.

Q: La cafeteria tenia 23 manzanas. Si ellos usaron 20 para preparar el almuerzo y luego compraron 6 manzanas mas. ¿Cuántas manzanas tiene la cafeteria?
""", max_new_tokens=10, temperature=0.7)
print(output[0]["generated_text"])


Q: Roger tiene 5 pelotas. El compra 2 cajas de pelotas. Cada caja tiene 3 pelotas. ¿Cuántos pelotas tiene Roger?
A: La respuesta es 11.

Q: La cafeteria tenia 23 manzanas. Si ellos usaron 20 para preparar el almuerzo y luego compraron 6 manzanas mas. ¿Cuántas manzanas tiene la cafeteria?
A: La respuesta es 47.

Q


In [15]:
output = generator("""
Q: Roger tiene 5 pelotas. El compra 2 cajas de pelotas. Cada caja tiene 3 pelotas. ¿Cuántos pelotas tiene Roger?
A: Roger empezo con 5 pelotas. 2 cajas de 3 pelotas cada una son 6 pelotas. 5 + 6 = 11. La respuesta es 11.

Q: La cafeteria tenia 23 manzanas. Si ellos usaron 20 para preparar el almuerzo y luego compraron 6 manzanas mas. ¿Cuántas manzanas tiene la cafeteria?
""", max_new_tokens=60, temperature=0.7)
print(output[0]["generated_text"])


Q: Roger tiene 5 pelotas. El compra 2 cajas de pelotas. Cada caja tiene 3 pelotas. ¿Cuántos pelotas tiene Roger?
A: Roger empezo con 5 pelotas. 2 cajas de 3 pelotas cada una son 6 pelotas. 5 + 6 = 11. La respuesta es 11.

Q: La cafeteria tenia 23 manzanas. Si ellos usaron 20 para preparar el almuerzo y luego compraron 6 manzanas mas. ¿Cuántas manzanas tiene la cafeteria?
A: El total de manzanas que tenían antes de comprar las más fue 23 - 20 = 3. Una vez que se compró más, quedan 3 + 6 = 9 manzanas. La respuesta es 9.

Q: Un re
